In [407]:
import pandas as pd
import numpy as np

In [408]:
players = pd.read_csv("./stats_with_clusters.csv")
lineups = pd.read_csv("./lineups21.csv")

In [409]:
lineups[['p1','p2','p3','p4','p5']]=lineups[' GROUP_NAME'].str.split(r'\s-\s',expand=True, n=4)
lineups.head()

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,...,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PIE_RANK,p1,p2,p3,p4,p5
0,Lineups,-201145-203115-203932-203999-1628420-,J. Green - W. Barton - A. Gordon - N. Jokic -...,1610612743,DEN,41,26,15,0.634,761.0,...,6370,10452,5593,5677,6814,J. Green,W. Barton,A. Gordon,N. Jokic,M. Morris
1,Lineups,-101108-203109-1626164-1628969-1629028-,C. Paul - J. Crowder - D. Booker - M. Bridges...,1610612756,PHX,38,31,7,0.816,754.0,...,9261,9045,6230,6504,7014,C. Paul,J. Crowder,D. Booker,M. Bridges,D. Ayton
2,Lineups,-201144-202711-203497-1626220-1628378-,M. Conley - B. Bogdanovic - R. Gobert - R. O'...,1610612762,UTA,42,29,13,0.690,666.0,...,6570,10529,5726,5990,7177,M. Conley,B. Bogdanovic,R. Gobert,R. O'Neale,D. Mitchell
3,Lineups,-201143-203935-1627759-1628369-1629057-,A. Horford - M. Smart - J. Brown - J. Tatum -...,1610612738,BOS,34,27,7,0.794,443.0,...,5935,10439,5608,6078,5394,A. Horford,M. Smart,J. Brown,J. Tatum,R. Williams III
4,Lineups,-201976-1626156-1626157-1629020-1630162-,P. Beverley - D. Russell - K. Towns - J. Vand...,1610612750,MIN,36,22,14,0.611,439.0,...,6434,10517,6238,6743,7262,P. Beverley,D. Russell,K. Towns,J. Vanderbilt,A. Edwards


In [410]:
players[['FirstName','LastName']]=players['Player'].str.split(expand=True,n=1)
players.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,SECONDARY_AST,POTENTIAL_AST,AST_POINTS_CREATED,AST_ADJ,AST_TO_PASS_PCT_ADJ,PCA-cluster,tSNE-cluster,UMAP-cluster,FirstName,LastName
0,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,...,0.2,2.1,2.8,1.4,0.061,0,6,4,Precious,Achiuwa
1,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,...,0.1,4.9,7.7,3.6,0.119,5,6,4,Steven,Adams
2,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,...,0.4,7.1,8.9,3.9,0.088,5,3,1,Bam,Adebayo
3,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,...,0.1,1.5,1.8,0.8,0.083,7,5,8,Santi,Aldama
4,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,...,0.2,2.0,2.4,1.1,0.049,5,6,4,LaMarcus,Aldridge


In [411]:
lineups[' TEAM_ABBREVIATION'] = lineups[' TEAM_ABBREVIATION'].str.strip()
np.setdiff1d(lineups[' TEAM_ABBREVIATION'].unique(),players['Tm'].unique())
#BKN, CHA, and PHX are different

array(['BKN', 'CHA', 'PHX'], dtype=object)

In [412]:
lineups.loc[lineups[' TEAM_ABBREVIATION']=='BKN',' TEAM_ABBREVIATION'] = 'BRK'
lineups.loc[lineups[' TEAM_ABBREVIATION']=='CHA',' TEAM_ABBREVIATION'] = 'CHO'
lineups.loc[lineups[' TEAM_ABBREVIATION']=='PHX',' TEAM_ABBREVIATION'] = 'PHO'

In [413]:
np.setdiff1d(lineups[' TEAM_ABBREVIATION'].unique(),players['Tm'].unique()) #now they match

array([], dtype=object)

We'll have to figure out how to deal with players on team 'TOT' separately

In [414]:
lineups[['p1','p2','p3','p4','p5']] = lineups[['p1','p2','p3','p4','p5']].applymap(lambda x: x.strip())

In [415]:
def player_lookup(row):
    pca_cluster = players.loc[(players['LastName'] == row[COL].split()[1]) & 
                              (players['Tm'] == row[' TEAM_ABBREVIATION']), 'PCA-cluster']
    
    #Check if the list is empty
    if pca_cluster.empty:
        ## plyaer has longer last name with JR. or III
        if (len(row[COL].split()) > 2):
            pca_cluster = players.loc[(players['LastName'] == (row[COL].split()[1]+" " +row[COL].split()[2])) & 
                                  (players['Tm'] == row[' TEAM_ABBREVIATION']), 'PCA-cluster']
            if pca_cluster.empty:
                pca_cluster = players.loc[(players['LastName'] == (row[COL].split()[1]+" " +row[COL].split()[2])) & 
                                  (players['Tm'] == 'TOT'), 'PCA-cluster']    
    
    #check if still empty
    if pca_cluster.empty:
        pca_cluster = players.loc[(players['LastName'] == row[COL].split()[1]) &
                                  (players['Tm'] == 'TOT'), 'PCA-cluster']
        if (len(pca_cluster) == 1): #if only one matching last name
            return pca_cluster.values[0]
        else:
            return "None"
        
        
    
    # Return the value
    return pca_cluster.values[0]

In [416]:
#permute through each column to match players
COL = 'x'
for col in ['p1','p2','p3','p4','p5']:
    COL = col
    lineups[col+'_cluster']=lineups.apply(player_lookup,axis=1)


In [422]:
#manually fix remaning "None"
for col in ['p1','p2','p3','p4','p5']:
    lineups.loc[(lineups[col]=='J. Holiday') & 
                (lineups[' TEAM_ABBREVIATION']!= 'MIL'),col+'_cluster'] = 0
    lineups.loc[lineups[col]=='A. Holiday',col+'_cluster'] = 6
    lineups.loc[lineups[col]=='I. Smith',col+'_cluster'] = 6
    lineups.loc[lineups[col]=='X. Tillman',col+'_cluster'] = 2
    lineups.loc[lineups[col]=='K. Johnson',col+'_cluster'] = 6
    lineups.loc[lineups[col]=='A. Johnson',col+'_cluster'] = 2
    lineups.loc[(lineups[col]=='J. Jackson') & 
                (lineups[' TEAM_ABBREVIATION']== 'SAC'),col+'_cluster'] = 7
    lineups.loc[lineups[col]=='J. Smith',col+'_cluster'] = 2
    

In [423]:
count = 0
for col in lineups.columns:
    count += (lineups[col] == 'None').sum()
count

0

In [424]:
lineups[lineups[' GROUP_NAME'].str.contains("D. Mitchell")]

,GROUP_SET,GROUP_ID,GROUP_NAME,TEAM_ID,TEAM_ABBREVIATION,GP,W,L,W_PCT,MIN,...,p1,p2,p3,p4,p5,p1_cluster,p2_cluster,p3_cluster,p4_cluster,p5_cluster
2,Lineups,-201144-202711-203497-1626220-1628378-,M. Conley - B. Bogdanovic - R. Gobert - R. O'...,1610612762,UTA,42,29,13,0.690,666.0,...,M. Conley,B. Bogdanovic,R. Gobert,R. O'Neale,D. Mitchell,3,3,5,0,1
32,Lineups,-202355-202711-203903-1626220-1628378-,H. Whiteside - B. Bogdanovic - J. Clarkson - ...,1610612762,UTA,45,29,16,0.644,220.0,...,H. Whiteside,B. Bogdanovic,J. Clarkson,R. O'Neale,D. Mitchell,5,3,3,0,1
67,Lineups,-202355-202711-204060-1626220-1628378-,H. Whiteside - B. Bogdanovic - J. Ingles - R....,1610612762,UTA,24,18,6,0.750,125.0,...,H. Whiteside,B. Bogdanovic,J. Ingles,R. O'Neale,D. Mitchell,5,3,6,0,1
74,Lineups,-203084-203200-1626168-1627745-1630558-,H. Barnes - J. Holiday - T. Lyles - D. Jones ...,1610612758,SAC,11,5,6,0.455,117.0,...,H. Barnes,J. Holiday,T. Lyles,D. Jones,D. Mitchell,3,0,0,2,6
128,Lineups,-202711-203497-203903-1626220-1628378-,B. Bogdanovic - R. Gobert - J. Clarkson - R. ...,1610612762,UTA,30,20,10,0.667,83.0,...,B. Bogdanovic,R. Gobert,J. Clarkson,R. O'Neale,D. Mitchell,3,5,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863,Lineups,-203090-1626158-1629056-1630169-1630558-,M. Harkless - R. Holmes - T. Davis - T. Halib...,1610612758,SAC,3,1,2,0.333,13.0,...,M. Harkless,R. Holmes,T. Davis,T. Haliburton,D. Mitchell,7,5,0,1,6
1878,Lineups,-1626158-1628368-1629056-1630169-1630558-,R. Holmes - D. Fox - T. Davis - T. Haliburton...,1610612758,SAC,2,2,0,1.000,13.0,...,R. Holmes,D. Fox,T. Davis,T. Haliburton,D. Mitchell,5,1,0,1,6
1941,Lineups,-202355-202711-203903-1627863-1628378-,H. Whiteside - B. Bogdanovic - J. Clarkson - ...,1610612762,UTA,3,2,1,0.667,12.0,...,H. Whiteside,B. Bogdanovic,J. Clarkson,D. House Jr.,D. Mitchell,5,3,3,0,1
1975,Lineups,-1629663-1629674-1630186-1630218-1630558-,L. King - N. Queta - J. Ramsey - R. Woodard I...,1610612758,SAC,2,0,2,0.000,12.0,...,L. King,N. Queta,J. Ramsey,R. Woodard II,D. Mitchell,7,2,7,4,6


In [421]:
#lineups.to_csv("./lineups_with_clusters.csv",index=False)